<a href="https://colab.research.google.com/github/HenryMakerCH/so-vits-svc-notebooks/blob/main/sovits4%E8%AE%AD%E7%BB%83%E6%8E%A8%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **注意**
## **第一次使用该笔记本时，确保你的谷歌硬盘内没有 `sovits4data` 文件夹**
## **否则将导致文件夹冲突**
## **文件夹将被用来存储必要文件** 
## **你可以修改 `sovits_data_dir` 变量来更改文件夹存储位置**

# **配置环境**

In [ ]:
#@title 查看GPU信息


!nvidia-smi

Tue Mar 14 15:07:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    33W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
#@title 克隆sovits4库并安装需求环境


!git clone https://github.com/svc-develop-team/so-vits-svc -b 4.0
%cd /content/so-vits-svc
!pip install --upgrade pip setuptools numpy numba
!pip install pyworld praat-parselmouth fairseq

Cloning into 'so-vits-svc'...
remote: Enumerating objects: 452, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 452 (delta 23), reused 48 (delta 20), pack-reused 400
Receiving objects: 100% (452/452), 8.10 MiB | 21.06 MiB/s, done.
Resolving deltas: 100% (196/196), done.
/content/so-vits-svc
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 84.9 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 63.4.3
    Uninstalling setuptools-63.4.3:
      Successfully uninstalled setuptools-63.4.3
  Attempting uninstall: pip
    Found existing inst

In [2]:
#@title 装载谷歌硬盘 选择文件夹位置


from google.colab import drive
drive.mount("/content/drive")

#@markdown 存储 **重要文件** 的文件夹，别忘了后面的斜杠 `/`
sovits_data_dir = "/content/drive/MyDrive/sovits4data/"  #@param {type:"string"}
RAW_DIR = sovits_data_dir + "raw/"
RESULTS_DIR = sovits_data_dir + "results/"
FILELISTS_DIR = sovits_data_dir + "filelists/"
CONFIGS_DIR = sovits_data_dir + "configs/"
LOGS_DIR = sovits_data_dir + "logs/44k/"

#@markdown

#@markdown　### **强烈建议全部勾选**

#@markdown 同步 **输入音频** 和 **输出音频**
sync_raw_and_results = True  #@param {type:"boolean"}
if sync_raw_and_results:
  !mkdir -p {RAW_DIR}
  !mkdir -p {RESULTS_DIR}
  !rm -rf /content/so-vits-svc/raw
  !rm -rf /content/so-vits-svc/results
  !ln -s {RAW_DIR} /content/so-vits-svc/raw
  !ln -s {RESULTS_DIR} /content/so-vits-svc/results

#@markdown 同步 **配置文件** 和 **模型**
sync_configs_and_logs = True  #@param {type:"boolean"}
if sync_configs_and_logs:
  !mkdir -p {FILELISTS_DIR}
  !mkdir -p {CONFIGS_DIR}
  !mkdir -p {LOGS_DIR}
  !rm -rf /content/so-vits-svc/filelists
  !rm -rf /content/so-vits-svc/configs
  !rm -rf /content/so-vits-svc/logs/44k
  !ln -s {FILELISTS_DIR} /content/so-vits-svc/filelists
  !ln -s {CONFIGS_DIR} /content/so-vits-svc/configs
  !ln -s {LOGS_DIR} /content/so-vits-svc/logs/44k


Mounted at /content/drive


In [3]:
#@title 获取必要模型


#@markdown ## [auspicious3000/contentvec](https://github.com/auspicious3000/contentvec)：

#@markdown #### 下载地址 [https://ibm.box.com/s/z1wgl1stco8ffooyatzdwsqn2psd9lrr](https://ibm.box.com/s/z1wgl1stco8ffooyatzdwsqn2psd9lrr)
#@markdown #### 手动上传到谷歌硬盘 /sovits4data/hubert/
#@markdown #### 没有下载直链
#@markdown **你下载的contentvec模型** 的存储文件夹, 别忘了后面的斜杠 `/`
sovits_data_dir = "/content/drive/MyDrive/sovits4data/hubert/"  #@param {type:"string"}
#@markdown **你下载的contentvec模型** 的文件名
contentvec_filename = "checkpoint_best_legacy_500.pt"  #@param {type:"string"}
CONTENTVEC_PATH = sovits_data_dir + contentvec_filename

!cp -v {CONTENTVEC_PATH} /content/so-vits-svc/hubert/checkpoint_best_legacy_500.pt

'/content/drive/MyDrive/sovits4data/hubert/checkpoint_best_legacy_500.pt' -> '/content/so-vits-svc/hubert/checkpoint_best_legacy_500.pt'


In [ ]:
#@title 获取预训练模型（强烈建议）


#@markdown - 预训练模型: `G_0.pth` `D_0.pth`
#@markdown   - 上传到谷歌硬盘 /sovits4data/logs/44k/

#@markdown 从svc-develop-team(不确定)或者其他什么地方搞

#@markdown Although the pretrained model generally does not cause any copyright problems, please pay attention to it. For example, ask the author in advance, or the author has indicated the feasible use in the description clearly.


# **数据集处理**

把打包后的数据传到 dataset_raw/

音频文件名不做要求

确保zip文件结构如下


```
yourzipfilename.zip
├───speaker0
│   ├───***.wav
│   ├───...
│   └───***.wav
└───speaker1
    ├───***.wav
    ├───...
    └───***.wav
```

In [ ]:
#@title 从谷歌硬盘读取数据集


#@markdown  **zip 文件**存储路径, 别忘了后面的斜杠 `/`
sovits_data_dir = "/content/drive/MyDrive/sovits4data/"  #@param {type:"string"}
#@markdown **你的 zip 文件名**，请勿命名为 "dataset.zip"
zip_filename = "henrymaker.zip"  #@param {type:"string"}
ZIP_PATH = sovits_data_dir + zip_filename

!unzip -od /content/so-vits-svc/dataset_raw {ZIP_PATH}

Archive:  /content/drive/MyDrive/sovits4data/henrymaker.zip
   creating: /content/so-vits-svc/dataset_raw/henrymaker/
  inflating: /content/so-vits-svc/dataset_raw/henrymaker/vocal_en_1.wav  
  inflating: /content/so-vits-svc/dataset_raw/henrymaker/vocal_en_10.wav  
  inflating: /content/so-vits-svc/dataset_raw/henrymaker/vocal_en_2.wav  
  inflating: /content/so-vits-svc/dataset_raw/henrymaker/vocal_en_3.wav  
  inflating: /content/so-vits-svc/dataset_raw/henrymaker/vocal_en_4.wav  
  inflating: /content/so-vits-svc/dataset_raw/henrymaker/vocal_en_5.wav  
  inflating: /content/so-vits-svc/dataset_raw/henrymaker/vocal_en_6.wav  
  inflating: /content/so-vits-svc/dataset_raw/henrymaker/vocal_en_7.wav  
  inflating: /content/so-vits-svc/dataset_raw/henrymaker/vocal_en_8.wav  
  inflating: /content/so-vits-svc/dataset_raw/henrymaker/vocal_en_9.wav  
  inflating: /content/so-vits-svc/dataset_raw/henrymaker/vocal_jp_1.wav  
  inflating: /content/so-vits-svc/dataset_raw/henrymaker/vocal_jp_1

In [ ]:
#@title 重采样到44100Hz

!python resample.py

./dataset_raw/henrymaker
36it [00:18,  1.93it/s]


In [ ]:
#@title 分割文件列表，生成配置文件

!python preprocess_flist_config.py

100% 1/1 [00:00<00:00, 475.65it/s]
Writing ./filelists/train.txt
100% 32/32 [00:00<00:00, 453438.27it/s]
Writing ./filelists/val.txt
100% 2/2 [00:00<00:00, 60787.01it/s]
Writing ./filelists/test.txt
100% 2/2 [00:00<00:00, 64035.18it/s]
Writing configs/config.json


In [ ]:
#@title 生成 hubert 和 f0

!python preprocess_hubert_f0.py

[36]
Loading hubert for content...
load model(s) from hubert/checkpoint_best_legacy_500.pt
INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX
INFO:fairseq.tasks.hubert_pretraining:current directory is /content/so-vits-svc
INFO:fairseq.tasks.hubert_pretraining:HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
INFO:fairseq.models.hubert.hubert:HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': transformer, 'dropout': 0.1, 'attention_dropout': 0.1, 'activati

In [ ]:
#@title 存储已处理数据集

#@markdown 把已处理过的数据集保存至谷歌硬盘以便下次训练

#@markdown **存储路径**, 别忘了后面的斜杠 `/`
sovits_data_dir = "/content/drive/MyDrive/sovits4data/henrymaker/" #@param {type:"string"}

#@markdown 你的谷歌硬盘内会有一个 `dataset.zip` 文件，内含 `dataset/` ，即为已处理数据集

!mkdir -p {sovits_data_dir}
!zip -r dataset.zip /content/so-vits-svc/dataset
!cp -vr dataset.zip "{sovits_data_dir}"

  adding: content/so-vits-svc/dataset/ (stored 0%)
  adding: content/so-vits-svc/dataset/44k/ (stored 0%)
  adding: content/so-vits-svc/dataset/44k/henrymaker/ (stored 0%)
  adding: content/so-vits-svc/dataset/44k/henrymaker/vocal_zh_8.wav.soft.pt (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/henrymaker/vocal_jp_506.wav.f0.npy (deflated 56%)
  adding: content/so-vits-svc/dataset/44k/henrymaker/vocal_jp_504.wav.f0.npy (deflated 50%)
  adding: content/so-vits-svc/dataset/44k/henrymaker/vocal_en_8.wav.soft.pt (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/henrymaker/vocal_jp_8.wav.f0.npy (deflated 58%)
  adding: content/so-vits-svc/dataset/44k/henrymaker/vocal_jp_1.wav.f0.npy (deflated 36%)
  adding: content/so-vits-svc/dataset/44k/henrymaker/vocal_jp_9.wav.f0.npy (deflated 40%)
  adding: content/so-vits-svc/dataset/44k/henrymaker/vocal_zh_8.wav.f0.npy (deflated 44%)
  adding: content/so-vits-svc/dataset/44k/henrymaker/vocal_jp_1.wav (deflated 9%)
  adding: content/so-v

# **训练**

In [4]:
#@title 从谷歌硬盘解压已有预处理数据集

#@markdown **你的已预处理数据集**的存储路径，别忘了后面的斜杠 `/`
sovits_data_dir = "/content/drive/MyDrive/sovits4data/" #@param {type:"string"}
CONFIG = sovits_data_dir + "configs/"
FILELISTS = sovits_data_dir + "filelists/"
DATASET = sovits_data_dir + "dataset.zip"

!cp -vr {CONFIG} /content/so-vits-svc/
!cp -vr {FILELISTS} /content/so-vits-svc/
!unzip {DATASET} -d /

cp: '/content/drive/MyDrive/sovits4data/configs/' and '/content/so-vits-svc/configs' are the same file
cp: '/content/drive/MyDrive/sovits4data/filelists/' and '/content/so-vits-svc/filelists' are the same file
Archive:  /content/drive/MyDrive/sovits4data/dataset.zip
   creating: /content/so-vits-svc/dataset/
   creating: /content/so-vits-svc/dataset/44k/
   creating: /content/so-vits-svc/dataset/44k/henrymaker/
  inflating: /content/so-vits-svc/dataset/44k/henrymaker/vocal_zh_8.wav.soft.pt  
  inflating: /content/so-vits-svc/dataset/44k/henrymaker/vocal_jp_506.wav.f0.npy  
  inflating: /content/so-vits-svc/dataset/44k/henrymaker/vocal_jp_504.wav.f0.npy  
  inflating: /content/so-vits-svc/dataset/44k/henrymaker/vocal_en_8.wav.soft.pt  
  inflating: /content/so-vits-svc/dataset/44k/henrymaker/vocal_jp_8.wav.f0.npy  
  inflating: /content/so-vits-svc/dataset/44k/henrymaker/vocal_jp_1.wav.f0.npy  
  inflating: /content/so-vits-svc/dataset/44k/henrymaker/vocal_jp_9.wav.f0.npy  
  inflating:

In [ ]:
#@title 开始训练

#@markdown 如果你要使用预训练模型，把它们上传到谷歌硬盘的 /sovits4data/logs/44k/

#@markdown 是否启用Tensorboard
tensorboard_on = False  #@param {type:"boolean"}

if tensorboard_on:
  %load_ext tensorboard
  %tensorboard --logdir logs/44k

!python train.py -c configs/config.json -m 44k

INFO:44k:{'train': {'log_interval': 100, 'eval_interval': 1000, 'seed': 1234, 'epochs': 10000, 'learning_rate': 0.0001, 'betas': [0.8, 0.99], 'eps': 1e-09, 'batch_size': 6, 'fp16_run': False, 'lr_decay': 0.999875, 'segment_size': 10240, 'init_lr_ratio': 1, 'warmup_epochs': 0, 'c_mel': 45, 'c_kl': 1.0, 'use_sr': True, 'max_speclen': 512, 'port': '8001', 'keep_ckpts': 3}, 'data': {'training_files': 'filelists/train.txt', 'validation_files': 'filelists/val.txt', 'max_wav_value': 32768.0, 'sampling_rate': 44100, 'filter_length': 2048, 'hop_length': 512, 'win_length': 2048, 'n_mel_channels': 80, 'mel_fmin': 0.0, 'mel_fmax': 22050}, 'model': {'inter_channels': 192, 'hidden_channels': 192, 'filter_channels': 768, 'n_heads': 2, 'n_layers': 6, 'kernel_size': 3, 'p_dropout': 0.1, 'resblock': '1', 'resblock_kernel_sizes': [3, 7, 11], 'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]], 'upsample_rates': [8, 8, 2, 2, 2], 'upsample_initial_channel': 512, 'upsample_kernel_sizes': [16, 16, 4

In [ ]:
#@title 训练聚类模型（可选）

#@markdown 详情见 [README.md#cluster-based-timbre-leakage-control](https://github.com/svc-develop-team/so-vits-svc#cluster-based-timbre-leakage-control)

!python cluster/train_cluster.py

# **推理**
### 从notebook中上传
### **或者**
### 手动上传到谷歌硬盘 /sovits4data/raw/

In [ ]:
#@title 上传wav文件，文件名不应含有特殊字符如 `#` `$` `(` `)`

%run wav_upload.py --type audio

In [ ]:
#@title 开始推理及下载

#@markdown 参数设置详见 [README.MD#inference](https://github.com/svc-develop-team/so-vits-svc#inference)

#@markdown

wav_filename = "LYTD_vocal.wav"  #@param {type:"string"}
model_filename = "G_800.pth"  #@param {type:"string"}
model_path = "/content/so-vits-svc/logs/44k/" + model_filename
speaker = "henrymaker"  #@param {type:"string"}
trans = "0"  #@param {type:"string"}
cluster_infer_ratio = "0"  #@param {type:"string"}

#@markdown

#@markdown 通常保持默认：
config_filename = "config.json"  #@param {type:"string"}
config_path = "/content/so-vits-svc/configs/" + config_filename
slice_db = "-40"  #@param {type:"string"}
wav_format = "flac"  #@param {type:"string"}
wav_output = "/content/so-vits-svc/results/" + wav_filename + "_" + trans + "key" + "_" + speaker + "." + wav_format

!python inference_main.py -n {wav_filename} -m {model_path} -c {config_path} -s {speaker} -t {trans} -cr {cluster_infer_ratio} -sd {slice_db} -wf {wav_format}

#@markdown

#@markdown 如果你不想直接从这里下载，请取消复选框
download_after_inference = True  #@param {type:"boolean"}

if download_after_inference:
  from google.colab import files
  files.download(wav_output)